## Import Dependencies

In [1]:
#data preprocessing
import pandas as pd
#produces a prediction model in the form of an ensemble of weak prediction models, typically decision tree
import xgboost as xgb
#the outcome (dependent variable) has only a limited number of possible values. 
#Logistic Regression is used when response variable is categorical in nature.
from sklearn.linear_model import LogisticRegression
#A random forest is a meta estimator that fits a number of decision tree classifiers 
#on various sub-samples of the dataset and use averaging to improve the predictive 
#accuracy and control over-fitting.
from sklearn.ensemble import RandomForestClassifier
#a discriminative classifier formally defined by a separating hyperplane.
from sklearn.svm import SVC
#displayd data
from IPython.display import display


#%matplotlib inline

/home/angloher/projects/Predicting_Winning_Teams/PredictEnv/local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import os
# Read data and drop redundant column.
loc = "/home/angloher/projects/Predicting_Winning_Teams/Datasets" #('Scotish_league',4),,('Bundesliga',2)
leagues=[('Turkish_league',1),('Premier_league',5),('La_Liga',2),('SerieA',2),('Bundesliga',2),('French_league',2),('Belgian_league',1),('Eredivisie',1),('Greek_league',1),('Portoguese_league',1)]



files=[]
betting_files=[]
num_last_games=20
num_last_seasons=6

for league,divisions in leagues:
    for i in range(1,divisions+1):
        files.append(pd.read_csv(os.path.join(loc,league,str(i),'Results','predict_final_dataset.csv')))
        betting_files.append(pd.read_csv(os.path.join(loc,league,str(i),'Results','predict_betting_odds.csv')))
data=pd.concat(files)
betting_data=pd.concat(betting_files)

data=pd.concat([data,betting_data],axis=1)

display(data.head())
#Full Time Result (H=Home Win, D=Draw, A=Away Win)
#HTGD - Home team goal difference
#ATGD - away team goal difference
#HTP - Home team points
#ATP - Away team points
#DiffFormPts Diff in points
#DiffLP - Differnece in last years prediction

#Input - 12 other features (fouls, shots, goals, misses,corners, red card, yellow cards)
#Output - Full Time Result (H=Home Win, D=Draw, A=Away Win) 

,Unnamed: 0,AC,ACC,ACS,AM10Conceded,AM10OppConceded,AM10OppPoints,AM10OppScored,AM10Scored,AM11Conceded,...,LY4HTGS,LY4HTP,MW,Unnamed: 0,B365H,B365D,B365A,HomeTeam,AwayTeam,Date
0,7524,NaN,4.90,6.20,1.0,1.8,0.8,1.0,1.0,1.0,...,0.000000,0.000000,21,7524,1.27,5.3,12.0,Buyuksehyr,Erzurum BB,2019-02-09
1,7525,NaN,5.95,3.95,1.0,1.6,1.9,2.3,2.0,1.0,...,1.212121,1.181818,21,7525,1.75,3.7,4.5,Rizespor,Antalyaspor,2019-02-09
2,7526,NaN,4.20,5.15,1.0,1.2,1.5,1.8,1.0,0.0,...,1.575758,1.848485,21,7526,1.65,3.8,5.2,Besiktas,Bursaspor,2019-02-09
3,7527,NaN,5.30,3.35,0.0,1.5,1.2,0.6,1.0,0.0,...,1.424242,1.242424,21,7527,1.85,3.4,4.5,Konyaspor,Yeni Malatyaspor,2019-02-10
4,7528,NaN,4.45,4.85,1.0,1.0,1.5,1.3,0.0,2.0,...,0.000000,0.000000,21,7528,2.35,3.3,3.1,Goztep,Alanyaspor,2019-02-10


## Preparing the Data

In [3]:
# Separate into feature set and target variable
#FTR = Full Time Result (H=Home Win, D=Draw, A=Away Win)

#train_set=data[pd.to_datetime(data.Date)<pd.to_datetime('{}-{}-{}'.format(train_val_split_date[0],train_val_split_date[1],train_val_split_date[2]))]
#val_set=data[pd.to_datetime(data.Date)>pd.to_datetime('{}-{}-{}'.format(train_val_split_date[0],train_val_split_date[1],train_val_split_date[2]))]
train_set=data
#print list(train_set.columns)

print len(train_set)

# Standardising the data.
from sklearn.preprocessing import scale
import math



#Center to the mean and component wise scale to unit variance.
cols = ['HTGS','HTGC','ATGS','ATGC','HTP','ATP','MW']#,
            #'HSS', 'ASS', 
            #'HSC', 'ASC', 'HSTS', 'ASTS', 'HSTC','ASTC', 
            #'HCS', 'ACS', 'HCC', 'ACC'] 

for i in range(1,num_last_games+1):
    cols.append('HM' + str(i)+'Scored')               
    cols.append('AM' + str(i)+'Scored')
    cols.append('HM' + str(i)+'Conceded')               
    cols.append('AM' + str(i)+'Conceded')
    cols.append('HM' + str(i)+'OppPoints')               
    cols.append('AM' + str(i)+'OppPoints')
    cols.append('HM' + str(i)+'OppScored')               
    cols.append('AM' + str(i)+'OppScored')
    cols.append('HM' + str(i)+'OppConceded')               
    cols.append('AM' + str(i)+'OppConceded')

for i in range(0,num_last_seasons-1):
    #cols.append('LY' + str(i)+'ACC')               
    #cols.append('LY' + str(i)+'HCC') 
    #cols.append('LY' + str(i)+'ACS')               
    #cols.append('LY' + str(i)+'HCS') 
    #cols.append('LY' + str(i)+'ASC')               
    #cols.append('LY' + str(i)+'HSC') 
    #cols.append('LY' + str(i)+'ASS')               
    #cols.append('LY' + str(i)+'HSS') 
    #cols.append('LY' + str(i)+'ASTC')               
    #cols.append('LY' + str(i)+'HSTC') 
    #cols.append('LY' + str(i)+'ASTS')               
    #cols.append('LY' + str(i)+'HSTS')
    cols.append('LY' + str(i)+'ATGC')               
    cols.append('LY' + str(i)+'HTGC') 
    cols.append('LY' + str(i)+'ATGS')               
    cols.append('LY' + str(i)+'HTGS') 
    cols.append('LY' + str(i)+'ATP')               
    cols.append('LY' + str(i)+'HTP') 
    cols.append('LY' + str(i)+'DivAT')               
    cols.append('LY' + str(i)+'DivHT')
    
for i in range(1,num_last_seasons*2-1):
    cols.append('LMUP' + str(i)+'HT')               
    cols.append('LMUP' + str(i)+'AT')    

betting_cols=['B365H','B365D','B365A']
for col in betting_cols:
    cols.append(col)
    
#for col in cols:
#    train_set = train_set[train_set[col].notnull()]


X_all = train_set.drop(['FTR'],1)
y_all = train_set['FTR']

betting_cols=['B365H','B365D','B365A','Date','Div','HomeTeam','AwayTeam']

for col in betting_cols:
    y_all=pd.concat([y_all,train_set[col]],axis=1)
    
print len(X_all),len(y_all)    
print list(X_all.columns),len(X_all.columns)
    
    

174
174 174
['Unnamed: 0', 'AC', 'ACC', 'ACS', 'AM10Conceded', 'AM10OppConceded', 'AM10OppPoints', 'AM10OppScored', 'AM10Scored', 'AM11Conceded', 'AM11OppConceded', 'AM11OppPoints', 'AM11OppScored', 'AM11Scored', 'AM12Conceded', 'AM12OppConceded', 'AM12OppPoints', 'AM12OppScored', 'AM12Scored', 'AM13Conceded', 'AM13OppConceded', 'AM13OppPoints', 'AM13OppScored', 'AM13Scored', 'AM14Conceded', 'AM14OppConceded', 'AM14OppPoints', 'AM14OppScored', 'AM14Scored', 'AM15Conceded', 'AM15OppConceded', 'AM15OppPoints', 'AM15OppScored', 'AM15Scored', 'AM16Conceded', 'AM16OppConceded', 'AM16OppPoints', 'AM16OppScored', 'AM16Scored', 'AM17Conceded', 'AM17OppConceded', 'AM17OppPoints', 'AM17OppScored', 'AM17Scored', 'AM18Conceded', 'AM18OppConceded', 'AM18OppPoints', 'AM18OppScored', 'AM18Scored', 'AM19Conceded', 'AM19OppConceded', 'AM19OppPoints', 'AM19OppScored', 'AM19Scored', 'AM1Conceded', 'AM1OppConceded', 'AM1OppPoints', 'AM1OppScored', 'AM1Scored', 'AM20Conceded', 'AM20OppConceded', 'AM20OppPo

In [4]:
import time
import numpy as np
#last 5 games for both sides

def test_letter(c):
    if not c.isdigit():
        return 4
    else:
        return c
    
def preprocess_features(X,cols):
    ''' Preprocesses the football data and converts catagorical variables into dummy variables. '''
    Divisions=['E0','E1','E2','E3','EC','D1','D2','F1','F2','SP1','SP2','B1','N1','G1','T1','P1','I1','I2']#,,'SC0','SC1''I1','I2''E0','E1','E2','E3','EC',
    cols.append('league')
    X['league']=None
    for div in Divisions:
        X[div[0:-1]]=0
        if div[0:-1] not in cols:
            cols.append(div[0:-1])
    for div in Divisions:
        X.ix[X['Div']==div,div[0:-1]]=1
        X.ix[X['Div']==div,'league']=float(test_letter(div[-1]))
    return X[cols]

val_cols=list(cols)
X_all=preprocess_features(X_all.copy(),val_cols)
#X_all=X_all[cols]

print list(X_all.columns),len(X_all.columns)

#start=time.time()


/home/angloher/projects/Predicting_Winning_Teams/PredictEnv/lib/python2.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


['HTGS', 'HTGC', 'ATGS', 'ATGC', 'HTP', 'ATP', 'MW', 'HM1Scored', 'AM1Scored', 'HM1Conceded', 'AM1Conceded', 'HM1OppPoints', 'AM1OppPoints', 'HM1OppScored', 'AM1OppScored', 'HM1OppConceded', 'AM1OppConceded', 'HM2Scored', 'AM2Scored', 'HM2Conceded', 'AM2Conceded', 'HM2OppPoints', 'AM2OppPoints', 'HM2OppScored', 'AM2OppScored', 'HM2OppConceded', 'AM2OppConceded', 'HM3Scored', 'AM3Scored', 'HM3Conceded', 'AM3Conceded', 'HM3OppPoints', 'AM3OppPoints', 'HM3OppScored', 'AM3OppScored', 'HM3OppConceded', 'AM3OppConceded', 'HM4Scored', 'AM4Scored', 'HM4Conceded', 'AM4Conceded', 'HM4OppPoints', 'AM4OppPoints', 'HM4OppScored', 'AM4OppScored', 'HM4OppConceded', 'AM4OppConceded', 'HM5Scored', 'AM5Scored', 'HM5Conceded', 'AM5Conceded', 'HM5OppPoints', 'AM5OppPoints', 'HM5OppScored', 'AM5OppScored', 'HM5OppConceded', 'AM5OppConceded', 'HM6Scored', 'AM6Scored', 'HM6Conceded', 'AM6Conceded', 'HM6OppPoints', 'AM6OppPoints', 'HM6OppScored', 'AM6OppScored', 'HM6OppConceded', 'AM6OppConceded', 'HM7Scored'

In [5]:
# Show the feature information by printing the first five rows

print "\nFeature values:"
print(X_all)
print X_all.dtypes


Feature values:
        HTGS      HTGC      ATGS      ATGC       HTP       ATP  MW  HM1Scored  \
0   1.550000  0.500000  0.900000  1.300000  2.200000  0.850000  21        3.0   
1   1.200000  1.450000  1.050000  1.450000  1.050000  1.400000  21        1.0   
2   1.900000  1.350000  0.950000  0.950000  1.650000  1.200000  21        6.0   
3   1.350000  1.050000  1.600000  1.100000  1.600000  1.650000  21        0.0   
4   1.000000  1.250000  0.900000  1.150000  1.100000  1.250000  21        0.0   
5   1.900000  1.100000  1.800000  1.350000  1.800000  1.650000  21        1.0   
6   0.850000  1.750000  1.900000  1.700000  1.000000  1.450000  21        0.0   
7   1.400000  1.350000  1.050000  1.750000  1.400000  0.850000  21        3.0   
0   1.000000  2.200000  1.960000  1.400000  0.680000  1.920000  26        0.0   
1   1.040000  1.320000  1.240000  1.520000  1.040000  1.280000  26        2.0   
2   2.240000  0.600000  1.480000  1.760000  2.480000  1.320000  26        1.0   
3   1.32000

In [6]:
X_all.fillna(0,inplace=True)
print len(X_all)


174


In [7]:

def split_train_target(y_train):
    matrix=y_train
    i=0
    #print matrix
    for train in y_train:
        #print train[0]
        if train[0]=='H':
            matrix[i][0]=0
        elif train[0]=='D':
            matrix[i][0]=1
        else:
            matrix[i][0]=2
        i+=1
    matrix=np.array(matrix,np.float)
    return matrix

def split_target(y_train):
    matrix=np.array([0]*len(y_train),np.float)
    i=0
    #print matrix
    for train in y_train:
        #print train[0]
        if train=='H':
            matrix[i]=0
        elif train=='D':
            matrix[i]=1
        else:
            matrix[i]=2
        i+=1
    return matrix


def predict(data_loader,model):
    probas=[]
    for i,data in enumerate(data_loader,0):
        inputs,labels =data
        inputs,labels=Variable(inputs),Variable(labels)
        pred=model(inputs)
        probas.append(pred.data.numpy())
        
    return probas

def accuracy(data_loader,model):
    summe=0
    count=0
    for i,data in enumerate(data_loader,0):
        inputs,labels =data
        inputs,labels=Variable(inputs),Variable(labels)
        pred=model(inputs)
        #print pred
        _,predicted=torch.max(pred.data,1)
        predicted=predicted.numpy()
        labels=labels.data.numpy()
        for i in range(len(predicted)):
            if labels[i]==predicted[i]:
                summe+=1# labels[i][predicted[i]]
        
        count+=len(predicted)
    
    return float(summe)/float(count)

In [8]:
import torch
import torch.utils.data
from torch.autograd import Variable
import matplotlib.pyplot as plt


#best_model=torch.load("/home/angloher/projects/Predicting_Winning_Teams/models/best_model.pt")

In [9]:
def decide_on_bet(probas,odds,thresh,tax,verbose,alpha,factor):
    exp_ret_home=probas[0]*odds[0]*alpha
    exp_ret_draw=probas[1]*odds[1]*alpha
    exp_ret_away=probas[2]*odds[2]*alpha
    bet=1
    win=0
    loss=0
    bet_count=0
    bet_amount=0
    #print odds
    #print exp_ret_home,exp_ret_draw,exp_ret_away
    if exp_ret_home>thresh and exp_ret_home > exp_ret_draw and exp_ret_home > exp_ret_away:
        bet_count+=1
        bet=((exp_ret_home-thresh))*factor
        bet_amount+=bet
        if verbose:
            print 'In {} {} vs {} I bet {} euros on Home with odds {} and expected return {}'.format(odds[5],odds[6],odds[8],bet,odds[0],exp_ret_home)
            B365odd=odds[0]/0.95
            if bet>100: 
                B365odd=odds[0]*100/bet/0.95
                print 'B365 odds need to be bigger than {} with (new odds * {} -1)*{}'.format(B365odd,probas[0],factor)
            else:    
                print 'B365 odds need to be bigger than {}'.format(B365odd)  
            
    if exp_ret_draw>thresh and exp_ret_draw > exp_ret_home and exp_ret_draw > exp_ret_away:
        bet_count+=1
        bet=((exp_ret_draw-thresh))*factor
        bet_amount+=bet
        if verbose:
            print 'In {} {} vs {} I bet {} euros on Draw with odds {} and expected return {}'.format(odds[5],odds[6],odds[8],bet,odds[1],exp_ret_draw)
            B365odd=odds[1]/0.95
            if bet>100: 
                B365odd=odds[1]*100/bet/0.95
                print 'B365 odds need to be bigger than {} with (new odds * {} -1)*{}'.format(B365odd,probas[1],factor)
            else:    
                print 'B365 odds need to be bigger than {}'.format(B365odd)  
    
    if exp_ret_away>thresh and exp_ret_away > exp_ret_draw and exp_ret_away > exp_ret_home:
        bet_count+=1
        bet=((exp_ret_away-thresh))*factor
        bet_amount+=bet
        if verbose:
            print 'In {} {} vs {} I bet {} euros on Away with odds {} and expected return {}'.format(odds[5],odds[6],odds[8],bet,odds[2],exp_ret_away)
            B365odd=odds[2]/0.95
            if bet>100: 
                B365odd=odds[2]*100/bet/0.95
                print 'B365 odds need to be bigger than {} with (new odds * {} -1)*{}'.format(B365odd,probas[2],factor)
            else:    
                print 'B365 odds need to be bigger than {}'.format(B365odd)  
            
    print ''  
    return bet_count,bet_amount
            

In [10]:
import glob,os


#prediction
predict_set=torch.utils.data.TensorDataset(torch.from_numpy(X_all.as_matrix()).float(),torch.FloatTensor(len(X_all),3).zero_())
predict_loader=torch.utils.data.DataLoader(predict_set,batch_size=len(X_all),shuffle=False,num_workers=8)


betting_data=y_all.drop(['FTR'],1).as_matrix()
print betting_data

best_models=[]
for file in glob.glob("/home/angloher/projects/Predicting_Winning_Teams/models/current_set/best*.pt"):
    best_models.append(torch.load(file))

print len(best_models)
thresh=1#np.linspace(0.5,2,10)
alpha=1
betting_tax=0
verbose=True
account_balance=7800
avg_bet=0.6
num_bets=60
factor=account_balance/200.0/avg_bet


probas=np.zeros((len(betting_data),3),np.float)    


for best_model in best_models:
    best_model.eval()
    #print predict(predict_loader,best_model)[0][3]
    probas+=predict(predict_loader,best_model)[0]
    #print probas
    
probas/=float(len(best_models))

print probas

bets_overall=0
amount_overall=0
list_of_bets=[]
        
for i in range(len(betting_data)):
    num_bets,amount=decide_on_bet(probas[i],betting_data[i],thresh,betting_tax,verbose,alpha,factor)
    amount_overall+=amount
    bets_overall+=num_bets
print bets_overall,amount_overall    

[[1.27 5.3 12.0 ..., 'Buyuksehyr' 'Erzurum BB' 'Erzurum BB']
 [1.75 3.7 4.5 ..., 'Rizespor' 'Antalyaspor' 'Antalyaspor']
 [1.65 3.8 5.2 ..., 'Besiktas' 'Bursaspor' 'Bursaspor']
 ..., 
 [1.15 8.0 15.0 ..., 'Benfica' 'Nacional' 'Nacional']
 [7.0 4.0 1.5 ..., 'Feirense' 'Sp Lisbon' 'Sp Lisbon']
 [2.2 3.0 3.7 ..., 'Setubal' 'Belenenses' 'Belenenses']]
1


/home/angloher/projects/Predicting_Winning_Teams/PredictEnv/local/lib/python2.7/site-packages/torch/serialization.py:316: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/angloher/projects/Predicting_Winning_Teams/PredictEnv/local/lib/python2.7/site-packages/torch/serialization.py:316: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/angloher/projects/Predicting_Winning_Teams/PredictEnv/local/lib/python2.7/site-packages/torch/serialization.py:316: SourceC

[[  9.99839187e-01   5.35658952e-19   1.11011622e-09]
 [  5.29686987e-01   2.56210251e-19   5.99802933e-12]
 [  9.99934196e-01   1.58273286e-21   1.13820488e-10]
 [  2.05434695e-01   5.85605457e-13   1.15815159e-02]
 [  3.49578186e-04   4.76475090e-01   6.60126889e-03]
 [  9.20949519e-01   8.81980604e-19   1.59053502e-06]
 [  1.06788753e-03   2.91039789e-13   1.15844095e-03]
 [  9.99812901e-01   2.53075910e-07   4.28674048e-06]
 [  6.86289975e-03   3.67732264e-06   8.56649280e-01]
 [  5.19910343e-02   5.64448932e-09   4.12775613e-02]
 [  9.74572539e-01   5.63283287e-08   9.94627044e-06]
 [  4.93714601e-01   4.84014273e-01   4.51552689e-01]
 [  3.11180775e-04   3.10555315e-08   9.17249501e-01]
 [  7.78689655e-03   2.46159580e-05   4.03307453e-02]
 [  5.43763787e-02   1.31421082e-04   4.30449191e-03]
 [  6.13428690e-02   5.76037671e-08   4.64346641e-08]
 [  1.79491900e-02   1.68287684e-21   6.41169393e-08]
 [  2.60831356e-01   9.66162980e-02   2.10676417e-01]
 [  7.16501713e-01   1.82907